In [1]:
from seglearn.features import SegFeatures, all_features, corr2
from seglearn.pipe import SegPipe
from seglearn.segment import Segment
from seglearn.datasets import load_watch

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
import matplotlib.pyplot as plt

In [2]:
data = load_watch()
print(data.keys())

fts = all_features()

feed = Pipeline([('segment', Segment()),('features', SegFeatures(fts))])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
y = data['y']
X = data['X']

pipe.fit(X, y)
print(pipe.f_labels)
ye, yp = pipe.predict(X, data['y'])
print(pipe.score(X, data['y']))

dict_keys(['X_labels', 'y_labels', 'y', 'side', 'X', 'subject'])
['skew_0', 'skew_1', 'skew_2', 'skew_3', 'skew_4', 'skew_5', 'mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'corr_0', 'corr_1', 'corr_2', 'corr_3', 'corr_4', 'corr_5', 'corr_6', 'corr_7', 'corr_8', 'corr_9', 'corr_10', 'corr_11', 'corr_12', 'corr_13', 'corr_14', 'std_0', 'std_1', 'std_2', 'std_3', 'std_4', 'std_5', 'min_0', 'min_1', 'min_2', 'min_3', 'min_4', 'min_5', 'kurt_0', 'kurt_1', 'kurt_2', 'kurt_3', 'kurt_4', 'kurt_5', 'max_0', 'max_1', 'max_2', 'max_3', 'max_4', 'max_5', 'mnx_0', 'mnx_1', 'mnx_2', 'mnx_3', 'mnx_4', 'mnx_5', 'hist4_0', 'hist4_1', 'hist4_2', 'hist4_3', 'hist4_4', 'hist4_5', 'hist4_6', 'hist4_7', 'hist4_8', 'hist4_9', 'hist4_10', 'hist4_11', 'hist4_12', 'hist4_13', 'hist4_14', 'hist4_15', 'hist4_16', 'hist4_17', 'hist4_18', 'hist4_19', 'hist4_20', 'hist4_21', 'hist4_22', 'hist4_23', 'mse_0', 'mse_1', 'mse_2', 'mse_3', 'mse_4', 'mse_5

In [1]:
from seglearn.features import SegFeatures, mean, var, std, skew
from seglearn.pipe import SegPipe
from seglearn.segment import Segment
from seglearn.datasets import load_watch
from seglearn.util import check_ts_data, ts_stats
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import numpy as np
data = load_watch()

check_ts_data(data['X'])
print(ts_stats(data['X'], data['y'], fs=50))

fts = {'mean': mean, 'var': var, 'std': std, 'skew': skew}
feed = Pipeline([('segment', Segment()),('features', SegFeatures(fts))])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
pipe.fit(data['X'], data['y'])
print(pipe.score(data['X'], data['y']))
      

{'total': {'Series_Time_Std': 8.757850351285423, 'N_TS_vars': 6, 'N_H_vars': 0, 'Series_Time_Range': (18.94, 52.36), 'Total_time': 4882.04, 'N_classes': 7, 'N_series': 140, 'Series_Time_Mean': 34.87171428571428}, 'by_class': {'Series_Time_Std': array([2.72084472, 8.0555543 , 8.77209872, 9.08521189, 7.98387775,
       6.09925209, 5.95709997]), 'Series_Time_Min': array([21.28, 22.48, 22.3 , 19.4 , 21.3 , 19.26, 18.94]), 'Class_labels': array([0, 1, 2, 3, 4, 5, 6]), 'Series_Time_Max': array([31.1 , 49.1 , 50.84, 52.36, 49.24, 45.12, 38.66]), 'Total_Time': array([532.44, 798.1 , 809.96, 747.9 , 752.08, 611.56, 630.  ]), 'Series_Time_Mean': array([26.622, 39.905, 40.498, 37.395, 37.604, 30.578, 31.5  ]), 'N_series': array([20, 20, 20, 20, 20, 20, 20])}}
0.9982895018174043


In [6]:
# univariate time series test and univariate feature test

def uv(X):
    X_new = [X[i][:,0] for i in np.arange(len(X))]
    return X_new


features = all_features()
del features['corr']
features = {'mean':mean}

feed = Pipeline([('segment', Segment()),('features', SegFeatures(features=features))])
est = RandomForestClassifier()
pipe = SegPipe(feed, est)
Xuv = uv(data['X'])

check_ts_data(Xuv)
print(ts_stats(Xuv, data['y']))

pipe = SegPipe(feed, est)
pipe.fit(Xuv, data['y'])
print(pipe.score(Xuv, data['y']))
print(pipe.f_labels)

{'by_class': {'Class_labels': array([0, 1, 2, 3, 4, 5, 6]), 'Series_Time_Max': array([1555., 2455., 2542., 2618., 2462., 2256., 1933.]), 'Series_Time_Min': array([1064., 1124., 1115.,  970., 1065.,  963.,  947.]), 'Total_Time': array([26622., 39905., 40498., 37395., 37604., 30578., 31500.]), 'Series_Time_Std': array([136.04223609, 402.77771475, 438.60493613, 454.26059426,
       399.19388773, 304.96260426, 297.85499828]), 'Series_Time_Mean': array([1331.1 , 1995.25, 2024.9 , 1869.75, 1880.2 , 1528.9 , 1575.  ]), 'N_series': array([20, 20, 20, 20, 20, 20, 20])}, 'total': {'Series_Time_Range': (947.0, 2618.0), 'N_H_vars': 0, 'Total_time': 244102.0, 'Series_Time_Std': 437.89251756427115, 'N_TS_vars': 1, 'N_classes': 7, 'Series_Time_Mean': 1743.5857142857142, 'N_series': 140}}
0.9435535599743425
['mean_0']


In [ ]:
np.mean()

In [3]:
print(score)

0.9818259568099209


In [15]:
fts = all_features()

In [17]:
all_features()

In [10]:
from seglearn.split import TemporalKFold
from seglearn.datasets import load_watch
data = load_watch()
splitter = TemporalKFold(n_splits=4)
X, y, cv = splitter.split(data['X'], data['y'])



560
